In [4]:
from model_utils import *
from morfeus_utils import *

In [5]:
# initialize the conformer tracker
monomers = np.arange(50)+1
combs = []
for comb in itertools.combinations(monomers, 2):
    combs.append(list(comb))
conf_tracker = dict(("_".join(map(str, el)), {"exp_file": "", "comp_file": []}) for el in combs)

# identify matches with experimental data
matches, conf_tracker = identify_matches(conf_tracker=conf_tracker)
avg_confs = np.mean(np.array([len(conf_tracker[key]) for key in list(conf_tracker.keys())]))
print(f"on average each complex has {round(avg_confs, 3)} conformers")

264 matching comp and exp spectra
on average each complex has 2.0 conformers


In [15]:
##### PREDICTING DIMER COMPUTATIONAL SPECTRUM FROM MONOMER SPECTRA AND FINGERPRINTS #####

# First load all complexes
master = pd.read_pickle("complexes_1168_boltz_weighted.pkl")
samples = master

# Load monomer data
data = load_monomer("comp_spectral_data/excited_state_monomer_spectra_csvs/", n_pts=64)

# Split into train and test sets 
training, testing = split_data(samples, test_size=0.25)

# Generate features from monomer spectra and fingerprints for both train and testing sets
training = gen_features(training, "comp_spectral_data/excited_state_monomer_spectra_csvs/", 
                        data, monomer_features=True, add_fps=True, fp_bits=256)
testing = gen_features(testing, "comp_spectral_data/excited_state_monomer_spectra_csvs/",
                       data, monomer_features=True, add_fps=True, fp_bits=256)

feature_size = len(training["training_features"].tolist()[0])
training, testing = shrink_spectrum(training, cut=1), shrink_spectrum(testing, cut=1)
target_size = len(training["comp_y"].tolist()[0])

print(f"training examples: {training.shape[0]}\ntesting examples: {testing.shape[0]}\nfeature size: {feature_size}\ntarget size: {target_size}")

training examples: 876
testing examples: 292
feature size: 640
target size: 311


In [16]:
# train a multitask GP to predict COMPUTATIONAL DIMER SPECTRUM from features
GPU = torch.cuda.is_available()
comp_model, comp_likelihood = train_multitask_GP(training, target="comp_y", n_iterations=300, GPU=GPU)
os.system("rm -rf after_training/")

# Run prediction and replace COMP spectra with prediction from MODEL
mean_train_preds, training = predict(comp_model, comp_likelihood, training, target="comp_y", train=True, interval=100, GPU=GPU)
mean_test_preds, testing = predict(comp_model, comp_likelihood, testing, target="comp_y", interval=50, GPU=GPU)

training["comp_y"] = mean_train_preds
testing["comp_y"] = mean_test_preds

**Using GPU**
Training iteration 50/500 --->  loss: -0.569
Training iteration 100/500 --->  loss: -0.766
Training iteration 150/500 --->  loss: -0.802
Training iteration 200/500 --->  loss: -0.821
Training iteration 250/500 --->  loss: -0.832
Training iteration 300/500 --->  loss: -0.840
Training iteration 350/500 --->  loss: -0.845
Training iteration 400/500 --->  loss: -0.848
Training iteration 450/500 --->  loss: -0.850
Training iteration 500/500 --->  loss: -0.851

Final Loss: -0.851

----- Running Predictions -----

	labeling 1/876, train: True
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GPU**
**Using GP

In [ ]:
# Rejoin training and testing DataFrames 
exp_samples = pd.concat([training, testing]).dropna()

# Split into train and test sets 
training, testing = split_data(exp_samples, test_size=0.25)

# Generate features from dimer spectra, monomer spectra, and fingerprints for both train and testing sets
training = gen_features(training, "comp_spectral_data/excited_state_monomer_spectra_csvs/", 
                        data, monomer_features=True, add_fps=True, fp_bits=128, comp_features=True)
testing = gen_features(testing, "comp_spectral_data/excited_state_monomer_spectra_csvs/",
                       data, monomer_features=True, add_fps=True, fp_bits=128, comp_features=True)
                       
feature_size = len(training["training_features"].tolist()[0])
target_size = len(training["exp_y"].tolist()[0])

print(f"training examples: {training.shape[0]}\ntesting examples: {testing.shape[0]}\nfeature size: {feature_size}\ntarget size: {target_size}")

In [ ]:
##### predicting EXPERIMENTAL DIMER SPECTRUM from PREDICTED COMP SPECTRA, MONOMER SPECTRA, and FPs #####
exp_model, exp_likelihood = train_multitask_GP(training, target="exp_y", n_iterations=250)

# Run predictions
mean_train_preds, training = predict(exp_model, exp_likelihood, training, target="exp_y", train=True, plot=True, compare=True)
mean_test_preds, testing = predict(exp_model, exp_likelihood, testing, target="exp_y", plot=True, compare=True)

In [ ]:
r2_after = pd.concat([testing["R2_after"], training["R2_after"]])
mae_after = pd.concat([testing["MAE_after"], training["MAE_after"]])